In [1]:
import tensorflow as tf
print(tf.__version__)

2.3.1


In [2]:
from sklearn.datasets import load_diabetes

diabetes_dataset = load_diabetes()

In [3]:
from sklearn.model_selection import train_test_split

data = diabetes_dataset['data']
targets = diabetes_dataset['target']

In [4]:
train_data, test_data, train_targets, test_targets = train_test_split(data, targets, test_size=0.1)

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = tf.keras.Sequential([
    Dense(128, activation='relu', input_shape=(train_data.shape[1],)),
    Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)
])

In [6]:
model.compile(loss='mse', optimizer="adam", metrics=['mae'])

In [7]:
class LossAndMetricCallback(tf.keras.callbacks.Callback):
    
    # Print the loss after every second batch in the training set
    def on_train_batch_end(self, batch, logs=None):
        if batch % 2 == 0:
            print(f"After batch {batch}, the loss is {logs['loss']:7.2f}")
            
    # Print the loss after each batch in the test set
    def on_test_batch_end(self, batch, logs=None):
        print('\n After batch {}, the loss is {:7.2f}.'.format(batch, logs['loss']))

    # Print the loss and mean absolute error after each epoch
    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {}: Average loss is {:7.2f}, mean absolute error is {:7.2f}.'.format(epoch, logs['loss'], logs['mae']))
    
    # Notify the user when prediction has finished on each batch
    def on_predict_batch_end(self,batch, logs=None):
        print("Finished prediction on batch {}!".format(batch))

In [8]:
# Train the model

history = model.fit(train_data, train_targets, epochs=20,
                    batch_size=100, callbacks=[LossAndMetricCallback()], verbose=False)

After batch 0, the loss is 29867.67
After batch 2, the loss is 29948.75
Epoch 0: Average loss is 29676.37, mean absolute error is  154.12.
After batch 0, the loss is 30945.99
After batch 2, the loss is 30142.36
Epoch 1: Average loss is 29548.61, mean absolute error is  153.73.
After batch 0, the loss is 29542.86
After batch 2, the loss is 28733.87
Epoch 2: Average loss is 29356.59, mean absolute error is  153.14.
After batch 0, the loss is 29369.72
After batch 2, the loss is 29444.87
Epoch 3: Average loss is 29089.23, mean absolute error is  152.31.
After batch 0, the loss is 27370.87
After batch 2, the loss is 29436.39
Epoch 4: Average loss is 28721.51, mean absolute error is  151.17.
After batch 0, the loss is 32752.90
After batch 2, the loss is 28186.91
Epoch 5: Average loss is 28252.03, mean absolute error is  149.65.
After batch 0, the loss is 26233.89
After batch 2, the loss is 27606.98
Epoch 6: Average loss is 27625.21, mean absolute error is  147.63.
After batch 0, the loss is 

In [9]:
# Evaluate the model

model_eval = model.evaluate(test_data, test_targets, batch_size=10, 
                            callbacks=[LossAndMetricCallback()], verbose=False)


 After batch 0, the loss is 16312.49.

 After batch 1, the loss is 13018.31.

 After batch 2, the loss is 15713.27.

 After batch 3, the loss is 13781.72.

 After batch 4, the loss is 14140.08.


In [10]:
# Get predictions from the model

model_pred = model.predict(test_data, batch_size=10,
                           callbacks=[LossAndMetricCallback()], verbose=False)

Finished prediction on batch 0!
Finished prediction on batch 1!
Finished prediction on batch 2!
Finished prediction on batch 3!
Finished prediction on batch 4!


### Application -- LRS Learning Rate Scheduler

In [18]:
# Define the learning rate scheduler function
lr_schedule = [
    (4, 0.03), (7, 0.02), (11, 0.005), (15, 0.007)
]

def get_new_epoch_lr(epoch, lr):
    epoch_in_sch = [i for i in range(len(lr_schedule)) if lr_schedule[i][0] == int(epoch)]
    if len(epoch_in_sch)> 0:
        return lr_schedule[epoch_in_sch[0]][1]
    else:
        return lr

In [19]:
# Define the custom callback

class LRScheduler(tf.keras.callbacks.Callback):
    
    def __init__(self, new_lr):
        super(LRScheduler, self).__init__()
        # Add the new learning rate function to our callback
        self.new_lr = new_lr

    def on_epoch_begin(self, epoch, logs=None):
        # Make sure that the optimizer we have chosen has a learning rate, and raise an error if not
        if not hasattr(self.model.optimizer, 'lr'):
              raise ValueError('Error: Optimizer does not have a learning rate.')
                
        # Get the current learning rate
        curr_rate = float(tf.keras.backend.get_value(self.model.optimizer.lr))
        
        # Call the auxillary function to get the scheduled learning rate for the current epoch
        scheduled_rate = self.new_lr(epoch, curr_rate)

        # Set the learning rate to the scheduled learning rate
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_rate)
        print('Learning rate for epoch {} is {:7.3f}'.format(epoch, scheduled_rate))

In [20]:
# Build the same model as before

new_model = tf.keras.Sequential([
    Dense(128, activation='relu', input_shape=(train_data.shape[1],)),
    Dense(64,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)        
])

In [21]:
# Compile the model

new_model.compile(loss='mse',
                optimizer="adam",
                metrics=['mae', 'mse'])

In [22]:
# Fit the model with our learning rate scheduler callback

new_history = new_model.fit(train_data, train_targets, epochs=20,
                            batch_size=100, callbacks=[LRScheduler(get_new_epoch_lr)], verbose=False)

Learning rate for epoch 0 is   0.001
Learning rate for epoch 1 is   0.001
Learning rate for epoch 2 is   0.001
Learning rate for epoch 3 is   0.001
Learning rate for epoch 4 is   0.030
Learning rate for epoch 5 is   0.030
Learning rate for epoch 6 is   0.030
Learning rate for epoch 7 is   0.020
Learning rate for epoch 8 is   0.020
Learning rate for epoch 9 is   0.020
Learning rate for epoch 10 is   0.020
Learning rate for epoch 11 is   0.005
Learning rate for epoch 12 is   0.005
Learning rate for epoch 13 is   0.005
Learning rate for epoch 14 is   0.005
Learning rate for epoch 15 is   0.007
Learning rate for epoch 16 is   0.007
Learning rate for epoch 17 is   0.007
Learning rate for epoch 18 is   0.007
Learning rate for epoch 19 is   0.007
